# **BILBIOTHEQUES**

In [1]:
import json

import pandas as pd
import numpy as np

import chess
import random

from Fct.fct_preprocess import *
from Fct.fct_eval import *

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

---

# **1. EXTRACTION DES DONNEES BRUTES**

Extraction des données depuis le fichier JSONL d'origine qui contient 137 000 000 positions différentes, chacune avec plusieurs évaluations stockfish . Nous limitons alors la taille de notre base de donnée a **500 000 positions**.

In [7]:
file_path = 'data\lichess_db_eval.jsonl'
data_raw = extract_data(file_path, max_rows=10000)
data_raw.head(5)

,fen,knodes,depth,cp,mate,line
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2
3,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,64.0,NaN,f7g7 e6e2 g7g6 d1c2 h8d8 e2d2 g6f6 a2a3 b7b5 c4a2
4,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,134.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4b3 a5a4 b3a2 c6d7 e6h6


La base de donnée est composée de **2 330 098 évaluations différentes** et **5 variables** : 

| Variable            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par stockfish pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |
| **line**        | Ligne (parfois appelée variante) désigne une séquence de coups qui découle d'une position donnée. Elle représente une possible continuation du jeu |

---

# **2. NETTOYAGE DES DONNEES**

Stockfish utilie différents parametre dans son algorythme, notamment le **nombre de noeuf (knodes)** et le la **profondeur (depth)**. En fonction de ces différents parametres, stockfish calcul le **centipions (cp)** et **l'évaluation du mate (mate)**.  

Après ces évaluations, stockfish peut alors parfois prédire des coups différents pour une même position, on appelle cela des **variantes**. Ces coups sont différents mais ont un impacte similaire ou quasi-similaire sur l'avantage qu'il donne a celui qui le joue (en terme d'evaluation de la position). 

Certains joueurs vont être plus a l'aise dans une variante plûtot qu'une autre en fonction de leurs styles de jeu ou de leurs connaissances, mais pour une IA cela n'as pas d'importance ! Toutefois, notre source de donnée nous propose plusieurs analyse de stockfish pour chaques position, apportant son lot de variante.

Nous pouvons alors utiliser 2 méthodes pour garder la meilleur variante.

### **Méthode N°1 : Analyse des Parametres et de l'Evaluation**
   1. **Profondeur de recherche (depth)** : Plus la profondeur est grande, plus l'analyse est précise. Priorisations des évaluations avec la profondeur maximale.

   2. **Centipions (cp) ou mat (mate)** : Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu. En absence de **mate**, choix d'évaluation avec la valeur **cp** la plus élevée pour le joueur actif

   3. **Nombre de nœuds analysés (knodes)** : Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

   4. **En cas d'égalité parfaite**: En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire.

### **Méthode N°2 : Coup le Plus Populaire** 
blablablabla


- Création d'une variable `best_move_m1`, qui extrait le prochain coup a jouer, c'est a dire le premier coup de la ligne.

In [8]:
data_V1 = extract_next_move(data_raw)
data_V1.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8


- Création d'une variable `best_move_m2` avec le coup le plus populaire par position en suivant la Méthode N°2

In [9]:
data_V2 = most_popular_predict_V1(data_V1)
data_V2.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8,f7g7


- Filtrage des lignes pour ne garder qu'une evaluation par position selon la méthode N°1.

In [10]:
data_V3 = filter_best_predict(data_V2)
data_V3.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,1B6/2pk1pp1/p6p/8/1P6/5P2/P1P3rP/1K6 w - -,44852,27,-643.0,NaN,h2h4 g2h2 c2c3 h2h4 a2a4 h6h5 b4b5 d7c8 b8a7 a6b5,h2h4,h2h4
1,1B6/2pk1pp1/p6p/8/1P6/5P2/P1P4r/K7 w - -,54193,26,-739.0,NaN,f3f4 d7c8 b8c7 c8c7 f4f5 h2c2 a1b1 c2e2 b1c1 e2e8,f3f4,f3f4
2,1B6/8/1NK5/k7/8/8/8/8 w - -,34280,75,NaN,17.0,b6d5 a5a4 c6c5 a4b3 d5b4 b3b2 b8f4 b2c3 f4e3 c3b3,b6d5,b6d5


- Suppression des variables inutiles. Les variables `knodes`, `depth`, `cp`, `mate` et `line` ne sont plus utile pour la suite du projet

In [11]:
data_cleaned = drop_usuless_columns(data_V3)
data_cleaned.head(5)

,fen,best_move_m1,best_move_m2
0,1B6/2pk1pp1/p6p/8/1P6/5P2/P1P3rP/1K6 w - -,h2h4,h2h4
1,1B6/2pk1pp1/p6p/8/1P6/5P2/P1P4r/K7 w - -,f3f4,f3f4
2,1B6/8/1NK5/k7/8/8/8/8 w - -,b6d5,b6d5
3,1B6/8/2K5/3N4/k7/8/8/8 w - -,c6c5,c6c5
4,1Bk5/2p2pp1/p6p/8/1P3P2/8/P1P4r/K7 w - -,b8a7,b8a7


- Sauvegarde des données netoyées en CSV

In [12]:
data_cleaned.to_csv('data/data_cleaned.csv')

---

# **3. ENTRAINEMENT DES MODELES**

blablablabla

- Chargement des Données

In [10]:
data = pd.read_csv('data/data_cleaned.csv' )

X = data['fen']
y = data['best_move_m1']

- Traduction des données sous une forme interpretable par le reseau de neurone.
  - **Position (Fen)** -> Matrice Numpy
  - **Coup a joué (UCI)** -> Encodage de chaque coup distincts (sous forme d'entier)

In [11]:
X = X.apply(fen_to_matrix)
X = np.array(X.tolist())

# Encode les mouvements et conversion en catégories
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))


# Sauvegarde de move_to_int dans un fichier JSON
with open('Models/move_int_dico.json', 'w') as file:
    json.dump(move_to_int, file)

### **Modèle N°1** :


   - **`Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12))`** : 
      - C'est une couche de convolution 2D qui extrait des caractéristiques spatiales de l'échiquier. 
      - **64 filtres** sont utilisés pour capter des motifs complexes, avec une taille de filtre de **3x3**, idéale pour des images petites comme un échiquier **8x8**. 
      - La fonction d'activation **ReLU** accélère l'apprentissage en gérant les gradients de manière efficace.

- **`Conv2D(128, (3, 3), activation='relu')`** : 
   - Une deuxième couche de convolution avec **128 filtres** permet de capturer des motifs plus complexes et abstraits sur l'échiquier.
   - La taille des filtres reste de **3x3**, et **ReLU** est utilisé pour améliorer l'apprentissage.

- **`Flatten()`** : 
   - Cette couche aplatit les sorties **2D** des couches précédentes en un vecteur **1D**, ce qui est nécessaire pour connecter les couches de convolution aux couches entièrement connectées.

- **`Dense(256, activation='relu')`** : 
   - Une couche entièrement connectée avec **256 neurones** pour capturer des relations non-linéaires complexes entre les caractéristiques extraites par les convolutions.
   - La fonction **ReLU** est utilisée pour une meilleure gestion des non-linéarités.

- **`Dense(len(move_to_int), activation='softmax')`** : 
   - La couche de sortie a un nombre de neurones égal au nombre de **coups possibles**. La fonction **Softmax** est utilisée pour classer les coups possibles en leur attribuant une probabilité, ce qui permet de sélectionner le coup le plus probable.

- **Définition du modèle**

In [17]:
model_1 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

model_1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 9, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1454)           │       373,678 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,632,110 (6.23 MB)

 Trainable params: 1,632,110 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

- **Entrainement du modèle**

blablablalba

In [18]:
model_1.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.0098 - loss: 6.9742 - val_accuracy: 0.0410 - val_loss: 4.9367
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0260 - loss: 6.2058 - val_accuracy: 0.0450 - val_loss: 4.4210
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0368 - loss: 5.7439 - val_accuracy: 0.0980 - val_loss: 4.2759
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.0667 - loss: 5.2250 - val_accuracy: 0.0900 - val_loss: 4.1638
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.1247 - loss: 4.5679 - val_accuracy: 0.1210 - val_loss: 4.1598
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.1937 - loss: 3.7821 - val_accuracy: 0.1160 - val_loss: 4.2880
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.3030 - loss: 2.9433 - val_accuracy: 0.1260 - val_loss: 4.3407
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.4275 - loss: 2.2326 - val_accu

- **Sauvegarde du Modèle**

In [19]:
model_1.save("Models/Modele_1_TF_25EPOCHS.keras")

### **Modèle N°2** :

Ajout de plus de couches de convolution permet de capturer des motifs encore plus complexes. Cependant, cela peut augmenter le risque de sur-apprentissage, donc il faudra ajuster les autres hyperparamètres (comme le taux d'apprentissage ou la régularisation).

- **Définition du modèle**

In [22]:
model_2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),  # Ajout d'une couche supplémentaire
    Flatten(),
    Dense(512, activation='relu'),  # Augmentation de la taille de la couche dense
    Dense(len(move_to_int), activation='softmax')
])

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 9, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 7, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │     1,835,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1454)           │       745,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,955,118 (11.27 MB)

 Trainable params: 2,955,118 (11.27 MB)

 Non-trainable params: 0 (0.00 B)

- **Entrainement du modèle**

In [23]:
model_2.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.0091 - loss: 6.9579 - val_accuracy: 0.0160 - val_loss: 4.9129
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.0243 - loss: 6.2549 - val_accuracy: 0.0770 - val_loss: 4.4433
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0290 - loss: 5.8038 - val_accuracy: 0.0740 - val_loss: 4.3501
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.0500 - loss: 5.3645 - val_accuracy: 0.1160 - val_loss: 4.2339
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.0824 - loss: 4.8405 - val_accuracy: 0.1190 - val_loss: 4.3054
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.1494 - loss: 4.1353 - val_accuracy: 0.1230 - val_loss: 4.2991
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.2346 - loss: 3.3744 - val_accuracy: 0.1240 - val_loss: 4.3689
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.3541 - loss: 2.5238 - val_accu

- **Sauvegarde du Modèle**

In [24]:
model_2.save("Models/Modele_2_TF_25EPOCHS.keras")

### **Modèle N°3** :

Moins de neurones dans la couche dense peut rendre le modèle plus rapide à entraîner et plus facile à régulariser, tout en conservant une bonne capacité de généralisation.

- Definition du Modèle

In [25]:
model_3 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),  # Dropout avec un taux de 50%
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

model_3.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_3.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 9, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1454)           │       373,678 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,632,110 (6.23 MB)

 Trainable params: 1,632,110 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

- Entrainement du Modèle

In [26]:
model_3.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.0070 - loss: 6.9705 - val_accuracy: 0.0550 - val_loss: 4.9903
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0251 - loss: 6.2771 - val_accuracy: 0.0480 - val_loss: 4.4623
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.0356 - loss: 5.7790 - val_accuracy: 0.0850 - val_loss: 4.3075
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.0614 - loss: 5.3763 - val_accuracy: 0.0910 - val_loss: 4.2171
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.0917 - loss: 4.8828 - val_accuracy: 0.1170 - val_loss: 4.0918
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.1336 - loss: 4.3178 - val_accuracy: 0.1090 - val_loss: 4.1464
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.1915 - loss: 3.7178 - val_accuracy: 0.1300 - val_loss: 4.1881
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.2743 - loss: 3.1034 - val_accu

- Sauvegarde du modèle

In [27]:
model_3.save("Models/Modele_3_TF_25EPOCHS.keras")

# **4. EVALUATION DES MODELES**

Partie entre deux random players

In [2]:
Game_result = play_game(random_player, random_player, print_game=False)

Résultat : 1/2-1/2


N Parties entre deux random players

In [60]:
N_Game_result = play_n_games(random_player, random_player, 1000)


Partie 1:
Résultat : 1-0

Partie 2:
Résultat : 1-0

Partie 3:
Résultat : 0-1

Partie 4:
Résultat : 1/2-1/2

Partie 5:
Résultat : 1/2-1/2

Partie 6:
Résultat : 1/2-1/2

Partie 7:
Résultat : 1/2-1/2

Partie 8:
Résultat : 1/2-1/2

Partie 9:
Résultat : 1/2-1/2

Partie 10:
Résultat : 1/2-1/2

Partie 11:
Résultat : 1/2-1/2

Partie 12:
Résultat : 1/2-1/2

Partie 13:
Résultat : 1/2-1/2

Partie 14:
Résultat : 1/2-1/2

Partie 15:
Résultat : 1/2-1/2

Partie 16:
Résultat : 1/2-1/2

Partie 17:
Résultat : 1/2-1/2

Partie 18:
Résultat : 1/2-1/2

Partie 19:
Résultat : 1/2-1/2

Partie 20:
Résultat : 1/2-1/2

Partie 21:
Résultat : 1/2-1/2

Partie 22:
Résultat : 1/2-1/2

Partie 23:
Résultat : 1/2-1/2

Partie 24:
Résultat : 1/2-1/2

Partie 25:
Résultat : 1/2-1/2

Partie 26:
Résultat : 1-0

Partie 27:
Résultat : 1/2-1/2

Partie 28:
Résultat : 1/2-1/2

Partie 29:
Résultat : 1/2-1/2

Partie 30:
Résultat : 1/2-1/2

Partie 31:
Résultat : 0-1

Partie 32:
Résultat : 1/2-1/2

Partie 33:
Résultat : 1/2-1/2

Parti

In [10]:
# Charge le model
from tensorflow.keras.models import load_model
model1 = load_model("Models\Modele_1_TF_25EPOCHS.keras")

# Chargement du dictionnaire sauvegardé pour une réutilisation ultérieure
with open('Models/move_int_dico.json', 'r') as file:
    move_int = json.load(file)

move_int_dico = {v: k for k, v in move_int.items()}

In [11]:
board = chess.Board()

In [12]:
predicted_moves = predict_next_move(board, model1, move_int_dico)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


In [13]:
def play_first_legal_move(board, predicted_moves_sorted):
    # Parcours des coups prédits
    for move in predicted_moves_sorted:
        # Vérifie si le coup est légal
        chess_move = chess.Move.from_uci(move)  # Convertit le coup en objet Move
        if chess_move in board.legal_moves:
            board.push(chess_move)  # Joue le coup légal
            return move  # Retourne le coup joué
    return None  # Si aucun coup n'est légal

In [48]:
move_played = play_first_legal_move(board, predicted_moves)
move_played
print(board)

r n . k . b n r
. . p . p p p p
p . . . . . . .
B . . p . q . .
. . . P . . . P
. . N B P N . .
P P P . . P R .
R Q . . K . . .
